# QUIZ - 2024-08

In [4]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import datetime, date
from service.models import *
from django.db.models import *
from django.db.models.functions import *
from zoneinfo import ZoneInfo

## PART 1 - CRUD Operations
1.1 สร้างบริการ (`Service`) รายละเอียดตามด้านล่าง และเพิ่มเข้าไปในหมวดหมู่ (`ServiceCategory`) ชื่อ "Hair Care"

- Name: "Hair Shower"
- Desciption: "Cold hair shower"
- Price: 100 (Provider 1) และ 200 (Provider 2)
- Service Provider: "Provider 1" และ "Provider 2"

*Hint: ดังนั้นต้องสร้าง 2 แถวในตาราง `Service`*


(0.5 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [3]:
p1 = ServiceProvider.objects.get(name="Provider 1")
p2 = ServiceProvider.objects.get(name="Provider 2")
s1 = Service.objects.create(
    service_provider = p1,
    name = "Hair Shower",
    description = "Cold hair shower",
    price = 100,
)
s2 = Service.objects.create(
    service_provider = p2,
    name = "Hair Shower",
    description = "Cold hair shower",
    price = 200,
)

c1 = ServiceCategory.objects.get(name="Hair Care")
c1.services.add(s1, s2)
c1.save()

In [4]:
# Check result
cat = ServiceCategory.objects.first()
cat.services.count()

6

1.2 ทำการแก้ไขราคาของ "Hair Shower" ของ "Provider 1" จาก 100 เป็น 300

(0.25 คะแนน)

In [9]:
# CODE HERE
p1 = Service.objects.get(name="Hair Shower", service_provider__name="Provider 1")
p1.price = 300
p1.save()

In [10]:
# Check result
services = Service.objects.filter(name="Hair Shower")
for service in services:
    print(f"Service Name: {service.name}, Provider Name: {service.service_provider.name}, Price: {service.price}")

Service Name: Hair Shower, Provider Name: Provider 2, Price: 200
Service Name: Hair Shower, Provider Name: Provider 1, Price: 300


1.3 ทำการแก้ไขนัดหมาย ณ วันที่ 2024-08-15 โดยเปลี่ยนจากเดิมที่นัดมาให้บริการ "Massage" ของ "Provider 3" เป็น "Hair Shower" ของ "Provider 1" แทน และทำการเลื่อนวันที่นัดหมายจาก 2024-08-15 เป็น 2024-08-18

(0.25 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [11]:
ap = Appointment.objects.get(
    appointment_date=date(2024,8,15), service__name="Massage", service__service_provider__name="Provider 3"
)

s = Service.objects.get(name="Hair Shower", service_provider__name="Provider 1")

ap.service = s
ap.appointment_date=date(2024,8,18)
ap.save()

In [12]:
# Check result
res = Appointment.objects.order_by("-appointment_date").first()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-08-18, Appoint Time: 11:30:00
Service Name: Hair Shower, Provider Name: Provider 1, Customer Name: Customer 1


1.4 ทำตามขั้นตอนดังนี้

1. สร้างผู้ให้บริการ (`ServiceProvider`) ชื่อ "Super Provider" ขึ้นมา (email: super@gmail.com, phone: 089-999-9999) และทำการเพิ่มบริการ (`Service`) ให้กับผู้ให้บริการนี้ (เทพมากสามารถทำได้ทุกบริการ) โดยทุกบริการราคาเท่ากับ 500 บาท

(1 คะแนน)

In [13]:
super = ServiceProvider.objects.create(
    name = "Super Provider",
    email = "super@gmail.com",
    phone = "089-999-9999"
)

In [14]:
services = [
    {"name": "Haircut", "description": "A simple haircut", "category": "Hair Care"},
    {"name": "Manicure", "description": "Nail trimming and polish", "category": "Nail Care"},
    {"name": "Massage", "description": "Full body massage", "category": "Body Care"},
    {"name": "Hair Shower", "description": "Cold hair shower", "category": "Hair Care"},
]

for service in services:
    cat = ServiceCategory.objects.get(name=service["category"])
    s = service.objects.create(
        service_provider = super,
        name = service["name"],
        description = service["description"],
        price = 500
    )
    cat.

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3461171226.py, line 12)

In [11]:
# Check result
for s in Service.objects.filter(service_provider__name="Super Provider"):
    print(f"Service Name: {s.name}, Price: {s.price}, Provider: {s.service_provider.name}")

Service Name: Haircut, Price: 500, Provider: Super Provider
Service Name: Manicure, Price: 500, Provider: Super Provider
Service Name: Massage, Price: 500, Provider: Super Provider
Service Name: Hair Shower, Price: 500, Provider: Super Provider


## PART 2 - Making Queries

สำหรับ PART 2 ให้ทำการ reset DB และ import ข้อมูลใน `service.sql` เข้าไปใหม่

2.1 ให้หาจำนวนนัดหมาย ที่นัดหมาย (`Appointment.appointment_date`) ในช่วงวันที่ 2024-08-05 ถึง 2024-08-10 ของ "Service Provider 1"

**Expected Output**

จะต้องมี 3 รายการ

(0.5 คะแนน)

In [2]:
# CODE HERE
Appointment.objects.filter(
    appointment_date__range=(date(2024,8,5), date(2024,8,10)), service__service_provider__name="Provider 1"
).count()


5

2.2 ให้หาว่ามีบริการใด ของผู้ให้บริการไหนบ้าง ที่มีการนัดหมายที่ถูกสร้าง (`Appointment.created_at`) ในช่วงวันที่ 2024-08-01 ถึง 2024-08-02

(0.5 คะแนน)

**Expected Output**

```
Name: Haircut, Provider: Provider 1
Name: Manicure, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2
```

In [8]:
# CODE HERE
dt1 = datetime(2024, 8, 1, tzinfo = ZoneInfo(key='Asia/Bangkok'))
dt2 = datetime(2024, 8, 2, tzinfo = ZoneInfo(key='Asia/Bangkok'))
ap = Appointment.objects.filter(created_at__range=(dt1, dt2)).order_by("service__service_provider__id")

for a in ap:
    print(f"Name: {a.service.name}, Provider: {a.service.service_provider.name}")

Name: Haircut, Provider: Provider 1
Name: Manicure, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2


2.3 ให้แสดงผลข้อมูลลูกค้า (`Customer`) ดังในตัวอย่าง

```python
[
    {
        "name": "Customer 4",
        "phone": "123-456-7893",
        "appointment_num": 3
    },
    {
        "name": "Customer 2",
        "phone": "123-456-7891",
        "appointment_num": 4
    },
    {
        "name": "Customer 3",
        "phone": "123-456-7892",
        "appointment_num": 3
    },
    {
        "name": "Customer 1",
        "phone": "123-456-7890",
        "appointment_num": 5
    }
]
```

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()` เท่านั้น การ loop และสร้าง dict เองจะไม่ได้คะแนน*

Hint: อยาก print dictionary สวยๆ ใช้ json.dumps

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

In [11]:
import json
ap = Appointment.objects.values(
    name = Concat("customer__name", Value("")), phone = Concat("customer__phone", Value(""))
).annotate(appointment_num = Count("id"))

print(json.dumps(list(ap), indent=4, sort_keys=False))

[
    {
        "name": "Customer 2",
        "phone": "123-456-7891",
        "appointment_num": 4
    },
    {
        "name": "Customer 3",
        "phone": "123-456-7892",
        "appointment_num": 3
    },
    {
        "name": "Customer 4",
        "phone": "123-456-7893",
        "appointment_num": 3
    },
    {
        "name": "Customer 1",
        "phone": "123-456-7890",
        "appointment_num": 5
    }
]


2.4 ให้แสดงข้อมูลผู้ให้บริการ (`ServiceProvider`) ดังนี้

*หมายเหตุ: Appointment Num หมายถึงจำนวนนัดหมายที่ผู้ให้บริการนั้นมีการได้รับนัด และ Appointed Service หมายถึงบริการที่มีการนัดหมายของผู้ให้บริการนั้นๆ*

(1 คะแนน)

**Expected Output**

```python
Name: Provider 1, Appointment Num: 7, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure, Massage, Massage
Name: Provider 2, Appointment Num: 5, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure
Name: Provider 3, Appointment Num: 2, Appointed Services: Manicure, Massage
Name: Provider 4, Appointment Num: 1, Appointed Services: Haircut
```

In [22]:
# CODE HERE
ap = Appointment.objects.values("service__service_provider__name").annotate(Appointment_num = Count("id")).order_by("-Appointment_num")

for p in ap:
    app_service = ', '.join(i.service.name for i in Appointment.objects.filter(service__service_provider__name=p['service__service_provider__name']))
    print(f"Name: {p["service__service_provider__name"]}, Appointment Num: {p["Appointment_num"]}, Appointed Services: {app_service}")

Name: Provider 1, Appointment Num: 7, Appointed Services: Haircut, Manicure, Manicure, Manicure, Massage, Massage, Massage
Name: Provider 2, Appointment Num: 5, Appointed Services: Haircut, Manicure, Manicure, Manicure, Massage
Name: Provider 3, Appointment Num: 2, Appointed Services: Manicure, Massage
Name: Provider 4, Appointment Num: 1, Appointed Services: Haircut


2.5 ให้หาว่าบริการ (`Service`) ไหนของผู้บริการไหนได้รับความนิยมสูงสุด (มีจำนวนนัดหมายมากที่สุด) และ นิยมน้อยที่สุด (มีจำนวนนัดหมายน้อยที่สุด)

(0.5 คะแนน)

**Expected Output**

```python
Name: Manicure, Provider: Provider 1, Appointment Num: 3
Name: Massage, Provider: Provider 1, Appointment Num: 3
Name: Manicure, Provider: Provider 2, Appointment Num: 3
Name: Massage, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 1, Appointment Num: 1
Name: Massage, Provider: Provider 2, Appointment Num: 1
Name: Manicure, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 4, Appointment Num: 1
Name: Haircut, Provider: Provider 2, Appointment Num: 1
Name: Massage, Provider: Provider 4, Appointment Num: 0
Name: Manicure, Provider: Provider 4, Appointment Num: 0
Name: Haircut, Provider: Provider 3, Appointment Num: 0
```

In [24]:
s = Service.objects.values("name", "service_provider").annotate(appointment_num=Count("appointment")).order_by("-appointment_num")
for i in s:
    print(f"Name: {i["name"]}, Provider: {i["service_provider"]}, Appointment Num: {i["appointment_num"]}")

Name: Manicure, Provider: 1, Appointment Num: 3
Name: Massage, Provider: 1, Appointment Num: 3
Name: Manicure, Provider: 2, Appointment Num: 3
Name: Massage, Provider: 3, Appointment Num: 1
Name: Haircut, Provider: 2, Appointment Num: 1
Name: Haircut, Provider: 1, Appointment Num: 1
Name: Haircut, Provider: 4, Appointment Num: 1
Name: Manicure, Provider: 3, Appointment Num: 1
Name: Massage, Provider: 2, Appointment Num: 1
Name: Haircut, Provider: 3, Appointment Num: 0
Name: Massage, Provider: 4, Appointment Num: 0
Name: Manicure, Provider: 4, Appointment Num: 0


2.6 ให้ดึงข้อมูลหมวดหมู่บริการโดยแปลงชื่อบริการ (`Service.name`) เป็นตัวใหญ่ (uppercase) และราคา (`Service.price`) เป็น 2 เท่า และเรียงลำดับตามราคา

(0.5 คะแนน)

**Expected Output**

```python
Name: HAIRCUT, Provider: Provider 1, New Price: $40
Name: MANICURE, Provider: Provider 1, New Price: $60
Name: MASSAGE, Provider: Provider 1, New Price: $200
Name: HAIRCUT, Provider: Provider 2, New Price: $40
Name: MANICURE, Provider: Provider 2, New Price: $60
Name: MASSAGE, Provider: Provider 2, New Price: $200
Name: HAIRCUT, Provider: Provider 3, New Price: $40
Name: MANICURE, Provider: Provider 3, New Price: $60
Name: MASSAGE, Provider: Provider 3, New Price: $200
Name: HAIRCUT, Provider: Provider 4, New Price: $40
Name: MANICURE, Provider: Provider 4, New Price: $60
Name: MASSAGE, Provider: Provider 4, New Price: $200
```

In [25]:
service = Service.objects.all()

for s in service:
    print(f"Name: {s.name.upper()}, Provider: {s.service_provider.name}, New Price: ${s.price * 2}")

Name: HAIRCUT, Provider: Provider 1, New Price: $40
Name: MANICURE, Provider: Provider 1, New Price: $60
Name: MASSAGE, Provider: Provider 1, New Price: $200
Name: HAIRCUT, Provider: Provider 2, New Price: $40
Name: MANICURE, Provider: Provider 2, New Price: $60
Name: MASSAGE, Provider: Provider 2, New Price: $200
Name: HAIRCUT, Provider: Provider 3, New Price: $40
Name: MANICURE, Provider: Provider 3, New Price: $60
Name: MASSAGE, Provider: Provider 3, New Price: $200
Name: HAIRCUT, Provider: Provider 4, New Price: $40
Name: MANICURE, Provider: Provider 4, New Price: $60
Name: MASSAGE, Provider: Provider 4, New Price: $200
